Import the libraries

In [172]:
#Import des librairies nécessaires.

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import torchvision
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_ResNet50_FPN_Weights
from torchvision import transforms as T

from sklearn.model_selection import train_test_split

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

J'ai fait le choix d'utiliser un modèle dont je ferai le finetuning.  C'est basé sur un CNN, donc très bien pour faire de l'analyse d'images et en particulier de la détection d'élements dans notre cas. : https://pytorch.org/vision/master/models/faster_rcnn.html
J'ai conscience que d'autres modèles existent, mais dans mon cas j'ai choisi le modèle le plus simple par soucis de performance.

In [173]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

Les boxes ont été crées avec labelimg. Le format des coordonées est celui de YOLO. 

In [174]:
# Import des Labels et des images 
import glob

data = []

# Get train labels
for file in glob.glob('imagesfull/labels/*.txt'):
    with open(file, 'r') as f:
        content = f.read()
        content = content.split(' ')
        if content[1]>content[3]:
            content[1], content[3] = content[3], content[1]
        if content[2]>content[4]:
            content[2], content[4] = content[4], content[2]
        
        filename = os.path.splitext(os.path.basename(file))[0]
        row = {'Image': filename, 'x1': content[1], 'y1': content[2], 'w': content[3], 'h': content[4].strip()}
        data.append(row)

df = pd.DataFrame(data)


#Conversion des coordonées format  Yolo Coordinates en coordoénes x1, y1, x2, y2 (format requis pour notre modèle) 
df["x1"] = pd.to_numeric(df["x1"])
df["y1"] = pd.to_numeric(df["y1"])
df["w"] = pd.to_numeric(df["w"])
df["h"] = pd.to_numeric(df["h"])

df["x2"] = df["x1"] + df["w"]/2
df["y2"] = df["y1"] + df["h"]/2

df["x1"] = df["x1"] - df["w"]/2
df["y1"] = df["y1"] - df["h"]/2

df.drop(columns=["w", "h"], inplace=True)


In [175]:
print(df.head())
print(df['Image'][0])

                                               Image        x1        y1  \
0  61055990_PHO_210115102717_36037262_BE578ZW_472...  0.096250  0.038334   
1  61098159_PHO_210621114136_36675117_DS228RZ_488...  0.187500  0.048334   
2  61113096_PHO_210906112642_37013023_BR381KB_496...  0.096355  0.115234   
3  61634454_PHO_210614153649_36641416_CW488EY_487...  0.106250  0.066667   
4  61635037_PHO_210617165130_36663833_EC496EA_488...  0.131250  0.001667   

         x2        y2  
0  0.616250  0.800001  
1  0.627500  0.770000  
2  0.675782  0.751953  
3  0.626250  0.743334  
4  0.785000  0.801667  
61055990_PHO_210115102717_36037262_BE578ZW_47242223


In [176]:
#Test to draw a rectangle on the image

im= Image.open('imagesfull/bis/images/'+df['Image'][3]+'.jpg')

draw = ImageDraw.Draw(im)
im_width, im_height = im.size
print(im.width, im.height)

x1 = 0.131250 * im_width
y1 = 0.001667 * im_height
x2 =  0.785000 * im_width
y2 = 0.801667*im_height

print(x1, y1, x2, y2)

draw.rectangle((x1, y1, x2, y2), outline='red')

# afficher l'image avec le rectangle dessiné
im.show()

800 600
105.0 1.0002 628.0 481.0002


In [177]:
#on récupère les images uniques qui nous servirons pour l'objet datasetCustom
unique_images = df.Image.unique()

df.to_csv('labels.csv', index=False)
print(unique_images)

['61055990_PHO_210115102717_36037262_BE578ZW_47242223'
 '61098159_PHO_210621114136_36675117_DS228RZ_488599412'
 '61113096_PHO_210906112642_37013023_BR381KB_496293520'
 '61634454_PHO_210614153649_36641416_CW488EY_48781857'
 '61635037_PHO_210617165130_36663833_EC496EA_488359414'
 '61637276_PHO_210705070511_36747110_FB170MK_490197018'
 '61638565_PHO_210719090259_36806831_CT701HG_491640312'
 '61865076_PHO_210521163156_36539817_FT857VF_485319017'
 '61865102_PHO_210531182130_36574497_FW453LE_486177017'
 '61876904_PHO_210723090635_36832542_CP966ZT_492277515'
 '62474067_PHO_210504101605_6105-851839482_DM-867-AT_fc2d6d93-f243-4825-8974-a52d520718c2'
 '62489478_PHO_210706183211_36758355_EZ044ZM_490533616'
 '62490594_PHO_210709111201_36774055_CJ431ZX_49090029' '911581721'
 '97705979_PHO_210514093024_36508533_AP832PZ_484531714'
 '97725165_PHO_210525084056_36544016_BY775AY_485388016'
 '97735324_PHO_210604160140_36599821_FM948RX_486806814'
 '97764167_PHO_210902093553_855952396_BA-252-ZK_1519418571'


Création de notre dataset customisé pour ensuite le donner à notre dataloader. 
Il faut donner notre dataframe contenant les images ainsi que les coordonées des boxes. Les images uniques serviront à les utiliser qu'une seule fois, et les indices pour lier les labels / box / images comme il faut. 

On retourne un tenseur compatible avec notre data loader


In [178]:

class CustomDataSet(torch.utils.data.Dataset):
    def __init__(self, df, unique_imgs , indices):
        self.df = df
        self.unique_imgs = unique_imgs
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        #Charger l'image
        img_name = self.unique_imgs[self.indices[idx]]
        boxes = self.df[self.df.Image == img_name].values[:, 1:].astype("float")
        img = Image.open("./imagesfull/images/" + img_name + ".jpg").convert("RGB")
        img = img.resize((800, 600), Image.LANCZOS)
        

        labels = torch.ones((1,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        return T.ToTensor()(img), target



In [179]:
#On sépare le jeu de données en un ensemle d'entrainement et un ensemble de test (80 / 20)
train_inds, test_inds = train_test_split(range(unique_images.shape[0]), test_size=0.2)

print(test_inds)

[29, 3, 19, 8, 24, 0, 36, 26]


Pour la fonction custom_collate, elle peut servir à traiter les éléments en lots avant de les donner au dataloader. Dans notre cas, nous avons pré traité les données avant, c'est pour cela que elle renvoie exactement ce que on lui donne. 

In [180]:
def custom_collate(data):
    return data

In [181]:
train_dl = torch.utils.data.DataLoader(CustomDataSet(df, unique_images, train_inds), 
                                       batch_size=2, 
                                       shuffle=True,
                                       collate_fn=custom_collate)

val_dl = torch.utils.data.DataLoader(CustomDataSet(df, unique_images, test_inds),
                                        batch_size=2,
                                        shuffle=True,
                                        collate_fn=custom_collate)


In [182]:
#Idéalement si l'on peut on utilise un GPU pour l'entrainement autrement ce sera le CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [183]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2  #2 classes : une pour les portes avants et une pour le background.
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) #On précise le nombre de classes pour finetuner le modèle

f:\Programmation\IA\TestBCA\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
f:\Programmation\IA\TestBCA\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [184]:
#Utilisation d'un optimiser de descente de gradient stochastique. Les paramètres sont à ajuster pour obtenir de meilleurs résultats. (De même pour les epochs)
optimiser = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
num_epochs = 5

In [185]:
#Phase d'entrainement du modèle. On donne epoch par epoch les images et les targets pour entrainer le modèle.
model.to(device)

for epoch in range(num_epochs):
    epoch_loss = 0
    for data in train_dl:
        imgs = []
        targets = []
        for d in data:
            imgs.append(d[0].to(device))
            targ = {}
            targ["boxes"] = torch.from_numpy(d[1]["boxes"]).to(device)
            targ["labels"] = d[1]["labels"].to(device)


            targets.append(targ)
        loss_dict = model(imgs, targets)
        loss = sum(v for v in loss_dict.values())
        epoch_loss += loss.cpu().detach().numpy()
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(epoch_loss)
    #Epoch loss nous permet de surveiller la perte d'entrainement du modèle. On s'attend à ce que cette perte diminue à chaque epoch.


10.402452134821242
3.7715503493643014
2.694903994624104
1.4930469409187002
1.4030803411622326


In [223]:
#Debut de la phase d'inférence. On évalue le modèle sur l'ensemble de test pour voir comment il se comporte sur des données qu'il n'a jamais vu.
model.eval()
data = iter(val_dl).__next__()

print(data[0][1])

{'boxes': array([[0.2396875, 0.0016665, 0.8428125, 0.9499995]]), 'labels': tensor([1])}


Pour évaluer le modèle nous faisons une inférence sur le df de val. Nous prenons 2 indicateurs qui sont : la similitude en % avec la boîte réelle, ainsi que le % de box qui sont supérieurs à un certain treshold. 

In [210]:
from sklearn.metrics import precision_score, recall_score, f1_score

model.eval()
val_targets = []
val_predictions = []

with torch.no_grad():
    for data_values in val_dl:
        imgs = [d[0].to(device) for d in data_values]
        targets = [d[1] for d in data_values]

        # Extraire les boîtes englobantes et les étiquettes pour chaque image
        for t in targets:
            targ = {}
            targ["boxes"] = torch.from_numpy(t["boxes"]).to(device)
            targ["labels"] = t["labels"].to(device)
            val_targets.append(targ["boxes"].cpu().numpy())

        # Effectuer une inférence sur le lot d'images
        prediction = model(imgs)

        # Extraire les boîtes englobantes prédites et les scores de confiance pour chaque image
        for i in range(len(prediction)):
            out_bbox1 = prediction[i]["boxes"].cpu()
            out_scores1 = prediction[i]["scores"].cpu()

            # On ne garde que la meilleure box
            keep = torchvision.ops.nms(out_bbox1, out_scores1, 0.7)

            if len(keep) > 0:
                best_box1 = out_bbox1[keep[0]]
                best_score = out_scores1[keep[0]]
                best_box1 = prediction[i]["boxes"][keep[0]].cpu().numpy()

                # Ajouter l'étiquette prédite à la liste val_predictions
                val_predictions.append(best_box1)
            else:
                # Ajouter une étiquette négative si aucune box n'est détectée
                val_predictions.append(np.zeros((4,)))


def compute_iou(box1, box2):
    # Calculer les coordonnées de l'intersection des deux boîtes
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Calculer l'aire de l'intersection des deux boîtes
    area_intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    # Calculer l'aire de chaque boîte
    area_box1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area_box2 = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # Calculer l'IoU
    iou = area_intersection / (area_box1 + area_box2 - area_intersection)

    # Convertir l'IoU en pourcentage
    iou_percent = iou * 100

    return iou_percent

val_targets = np.array(val_targets)


val_targets = np.squeeze(val_targets)
#resize val_targets pour que ce soit normalisé avec les prédictions.
for i in range(len(val_targets)):
    val_targets[i][0] = val_targets[i][0]*800
    val_targets[i][1] = val_targets[i][1]*600
    val_targets[i][2] = val_targets[i][2]*800
    val_targets[i][3] = val_targets[i][3]*600
results = []

for i in range(len(val_predictions)):
    result = compute_iou(val_targets[i], val_predictions[i])
    results.append(result)


results = np.array(results)
moyenne_similitude = np.mean(results)

print("En moyenne, les box prédites ont une similitude de {:.2f}% avec les box réelles.".format(moyenne_similitude))


treshold = 70
# Calculer le pourcentage de boîtes qui ont une similitude supérieure au treshold
nbr_boites_superieures = np.count_nonzero(results >= treshold)
pourcentage_boites_superieures = nbr_boites_superieures / len(results) * 100

print("{:.2f}% des boîtes ont une similitude supérieure à {:.2f}%d .".format(pourcentage_boites_superieures, treshold))
    

[array([3.6626103e-04, 0.0000000e+00, 2.0814431e+00, 1.9141083e+00],
      dtype=float32), array([0.00322129, 0.        , 1.6998055 , 2.298231  ], dtype=float32), array([0., 0., 0., 0.]), array([0.00413149, 0.        , 0.87406456, 1.6055486 ], dtype=float32), array([8.5678598e-04, 0.0000000e+00, 9.7575241e-01, 1.2119522e+00],
      dtype=float32), array([0., 0., 0., 0.]), array([1.3288165, 0.       , 1.7261846, 0.1723454], dtype=float32), array([0., 0., 0., 0.])]
[[190.5    150.0003 661.5    566.0001]
 [140.5     39.9999 463.5    455.9997]
 [191.75     0.9999 674.25   569.9997]
 [144.       0.9999 678.     452.9997]
 [ 85.      39.9999 501.     446.0001]
 [106.6664   2.2221 460.8328 443.3331]
 [ 77.      23.0001 493.     480.0003]
 [ 86.25     1.2    503.75   520.8   ]]
En moyenne, les box prédites ont une similitude de 0.00% avec les box réelles.
0.00% des boîtes ont une similitude supérieure à 70.00%d .


Ensuite nous faison une inférence simple pour afficher l'image et la box associée (sur une nouvelle image.)

In [224]:
#On récupère les images et les targets pour les afficher
img = data[0][0]
boxes = data[0][1]["boxes"]
labels = data[0][1]["labels"]

In [226]:
labels

tensor([1])

In [227]:
#on donne l'image à notre modèle.
output = model([img.to(device)])

In [230]:
output

[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([], grad_fn=<IndexBackward0>)}]

In [214]:
#On recupère les résultats de la prédiction (les boites englobantes et les scores associés)
out_bbox = output[0]["boxes"]
out_scores = output[0]["scores"]

out_scores

tensor([], grad_fn=<IndexBackward0>)

In [215]:
#Supprossion des boites qui se chevauchent si le IoU est supérieur à 0.7
#Je m'en sers pour trier les scores par indice décroissant
keep = torchvision.ops.nms(out_bbox, out_scores, 0.7)


In [217]:
out_bbox

tensor([], size=(0, 4), grad_fn=<StackBackward0>)

In [232]:
im = (img.permute(1, 2, 0).cpu().detach().numpy()*255).astype('uint8')

In [ ]:
im

array([[[194, 176, 156],
        [188, 170, 150],
        [194, 176, 156],
        ...,
        [ 71,  59,  35],
        [ 72,  60,  36],
        [ 71,  59,  35]],

       [[194, 176, 156],
        [188, 170, 150],
        [194, 176, 156],
        ...,
        [ 71,  59,  35],
        [ 72,  60,  36],
        [ 70,  58,  34]],

       [[195, 177, 157],
        [189, 171, 151],
        [194, 176, 156],
        ...,
        [ 71,  59,  35],
        [ 72,  60,  36],
        [ 70,  58,  34]],

       ...,

       [[251, 228, 212],
        [251, 228, 212],
        [250, 227, 211],
        ...,
        [178, 142, 128],
        [178, 142, 128],
        [178, 142, 128]],

       [[251, 228, 212],
        [251, 228, 212],
        [251, 228, 212],
        ...,
        [179, 146, 131],
        [176, 143, 128],
        [173, 140, 125]],

       [[252, 229, 213],
        [252, 229, 213],
        [251, 228, 212],
        ...,
        [181, 148, 133],
        [176, 143, 128],
        [171, 138, 123]]

In [234]:

# supposons que la forme de l'image d'origine est (hauteur, largeur, canaux)
hauteur, largeur, canal = im.shape
print(im.shape)

# On garde seulement la boîte qui à le meilleur score. Ou si il n'y a pas de box détectée, on crée une box vide.
try:
    best_box = out_bbox[keep[0]].cpu().detach().numpy()
except:
    best_box = np.zeros((4,))

# convertir les coordonnées normalisées en coordonnées entières
best_box_int = [int(best_box[0] * largeur), int(best_box[1] * hauteur), int(best_box[2] * largeur), int(best_box[3] * hauteur)]

# dessiner les rectangles avec des coordonnées entières
vsamlpe = Image.fromarray(im)
draw = ImageDraw.Draw(vsamlpe)
draw.rectangle(list(best_box_int), outline="red", width=3)
vsamlpe.show()



(600, 800, 3)
[0, 0, 0, 0]
